In [1]:
from collections import namedtuple
import itertools
import math
import enum

In [2]:
# Classes used for declaring the board
PointSet = namedtuple("PointSet", " points op val")

class op(enum.Enum):
    DIV = enum.auto()
    PROD = enum.auto()
    SUM = enum.auto()
    SUB = enum.auto()
    NULL = enum.auto()

In [3]:
# A 3x3 board
BOARD_SIZE=3

sets = [
    PointSet( ((0,0), (1,0)), op.DIV, 2 ),
    PointSet( ((2,0), (2,1)), op.SUB, 2 ),
    PointSet( ((0,1), (0,2)), op.PROD, 6 ),
    PointSet( ((1,1), (1,2), (2,2)), op.SUM, 6 )
]

sets

[PointSet(points=((0, 0), (1, 0)), op=<op.DIV: 1>, val=2),
 PointSet(points=((2, 0), (2, 1)), op=<op.SUB: 4>, val=2),
 PointSet(points=((0, 1), (0, 2)), op=<op.PROD: 2>, val=6),
 PointSet(points=((1, 1), (1, 2), (2, 2)), op=<op.SUM: 3>, val=6)]

In [4]:
# 4x4 board
BOARD_SIZE=4

sets = [
    PointSet( ((0,0), (0,1)), op.SUB, 1 ),
    PointSet( ((0,2), (0,3)), op.SUB, 3 ),
    PointSet( ((1,0), (1,1)), op.SUM, 3 ),
    PointSet( ((1,2), (2,2)), op.SUB, 2 ),
    PointSet( ((1,3), (2,3)), op.DIV, 2 ),
    PointSet( ((2,0), (2,1)), op.SUB, 1 ),
    PointSet( [(3,0)], op.NULL, 4 ), # Have to do this with single-cell sets to make this work
    PointSet( ((3,1), (3,2), (3,3)), op.PROD, 6 )
]

In [5]:
# Validate that every point is in a point set
set.union(*[set(s.points) for s in sets])

{(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3)}

In [6]:
# convenience func
def get_nums():
    return range(1, BOARD_SIZE+1)

# Set up a grid of possible values
possible_values = {}
for row in range(BOARD_SIZE):
    for col in range(BOARD_SIZE):
        possible_values[(row, col)] = set(get_nums())

possible_values


{(0, 0): {1, 2, 3, 4},
 (0, 1): {1, 2, 3, 4},
 (0, 2): {1, 2, 3, 4},
 (0, 3): {1, 2, 3, 4},
 (1, 0): {1, 2, 3, 4},
 (1, 1): {1, 2, 3, 4},
 (1, 2): {1, 2, 3, 4},
 (1, 3): {1, 2, 3, 4},
 (2, 0): {1, 2, 3, 4},
 (2, 1): {1, 2, 3, 4},
 (2, 2): {1, 2, 3, 4},
 (2, 3): {1, 2, 3, 4},
 (3, 0): {1, 2, 3, 4},
 (3, 1): {1, 2, 3, 4},
 (3, 2): {1, 2, 3, 4},
 (3, 3): {1, 2, 3, 4}}

In [7]:
# Pretty-printing the current state
def print_cell(nums):
    return "".join([str(i) if i in nums else " " for i in get_nums()])

def print_grid(values):
    for row in range(BOARD_SIZE):
        print ("|".join([print_cell(values[(row, col)]) for col in range(BOARD_SIZE)]))

print_grid(possible_values)


1234|1234|1234|1234
1234|1234|1234|1234
1234|1234|1234|1234
1234|1234|1234|1234


In [8]:
# For a PointSet, enumerate all the sets of numbers that could meet the arithmetic constraint
def enum_set_solutions(ptset):
    solutions = {}
    if ptset.op == op.NULL:
        solutions = [[ptset.val]]
    if ptset.op == op.DIV:
        solutions = [(i, i*ptset.val) for i in range(1, int(BOARD_SIZE/ptset.val)+1)]
    if ptset.op == op.SUB:
        solutions = [(i, i+ptset.val) for i in range(1, BOARD_SIZE-ptset.val+1)]
    if ptset.op == op.PROD:
        factors = (i for i in get_nums() if ptset.val%i == 0)
        combos = itertools.product(factors, repeat=len(ptset.points))
        possibles = [tuple(sorted(combo)) for combo in combos if math.prod(combo)==ptset.val]
        solutions = list(set(possibles))
    if ptset.op == op.SUM:
        possible_nums = get_nums()
        combos = itertools.product(possible_nums, repeat=len(ptset.points))
        possibles = [tuple(sorted(combo)) for combo in combos if sum(combo)==ptset.val]
        solutions = list(set(possibles))

    return solutions

In [9]:
[enum_set_solutions(s) for s in sets]

[[(1, 2), (2, 3), (3, 4)],
 [(1, 4)],
 [(1, 2)],
 [(1, 3), (2, 4)],
 [(1, 2), (2, 4)],
 [(1, 2), (2, 3), (3, 4)],
 [[4]],
 [(1, 2, 3)]]

In [10]:
# Apply the pointset math
# For each set, compute the possible values that can fill the set,
# and restrict the global possible values to those values
def do_pointset_math_logic():
    for s in sets:
        sols = enum_set_solutions(s)
        nums = set.union(*[set(sol) for sol in sols])
        for pt in s.points:
            possible_values[pt] = possible_values[pt].intersection(nums)

do_pointset_math_logic()
print_grid(possible_values)

1234|1234|1  4|1  4
12  |12  |1234|12 4
1234|1234|1234|12 4
   4|123 |123 |123 


In [11]:
# This does the pigeonhole logic of one number per row/column
# The logic is easy when a cell has only one possible value
# But it can be generalized; if a row has two pairs with two values, those two values can't be elsewhere
# Eg the ' 23' in two cells of row 0 means that there can't be a 2 or 3 elsewhere in row 0
# And so on for 3 sets of size 3, etc
# TODO generalize this to columns
def do_pigeonhole_logic():
    for row in range(BOARD_SIZE):
        for size in range (1,4):
            pairs = {col: possible_values[(row, col)] for col in range(BOARD_SIZE) if len(possible_values[(row, col)])==size}
            paired_pairs = [pair for pair in pairs.values() if list(pairs.values()).count(pair)==size]
            nums = []
            if paired_pairs:
                nums = set.union(*[set(pair) for pair in paired_pairs])
                for col in range(BOARD_SIZE):
                    possibles = possible_values[(row, col)]
                    if possibles not in paired_pairs:
                        possible_values[(row, col)] = possibles - nums

    # For now copy/paste for cols
    for col in range(BOARD_SIZE):
        for size in range (1,4):
            pairs = {row: possible_values[(row, col)] for row in range(BOARD_SIZE) if len(possible_values[(row, col)])==size}
            paired_pairs = [pair for pair in pairs.values() if list(pairs.values()).count(pair)==size]
            nums = []
            if paired_pairs:
                nums = set.union(*[set(pair) for pair in paired_pairs])
                for row in range(BOARD_SIZE):
                    possibles = possible_values[(row, col)]
                    if possibles not in paired_pairs:
                        possible_values[(row, col)] = possibles - nums

do_pigeonhole_logic()
print_grid(possible_values)

 23 | 23 |1  4|1   
12  |12  |  34|   4
123 |1234|1234|12  
   4|123 |123 |123 


In [12]:
# another iteration
do_pigeonhole_logic()
print_grid(possible_values)

 23 | 23 |   4|1   
12  |12  |  3 |   4
123 |1234|12  | 2  
   4|123 |12  | 23 


In [13]:
# a few more iterations
[do_pigeonhole_logic() for _ in range(3)]
print_grid(possible_values)

 2  |  3 |   4|1   
1   | 2  |  3 |   4
  3 |   4|1   | 2  
   4|1   | 2  |  3 


In [14]:
# TODO the pointset math logic is pretty basic now
# I could expand it to see which sets are actually possible based on current state
# E.g., if a valid set is {1, 2 ,3}, but we know that 3 can't be in any of the three cells,
# we can eliminate that set

# TODO iteratively run the pigeonhole and math logic until we solve the problem (or converge without a solution)